In [1]:
import json
import openai
from IPython.display import display, JSON

In [2]:
file_path = '../OPENAI_API_KEY.txt'
with open(file_path, 'r') as file:
    openai.api_key = file.readline()

def get_openai_model_price(model_name):
    if(model_name in 'gpt-4o-mini-2024-07-18'):
        return {'name': model_name, 'input_token_price_per_million_usd' : 0.15, 'cached_input_token_price_per_million_usd' : 0.075, 'output_token_price_per_million_usd' : 0.60}
    return None

def get_chat_completion_stats(response):
    response_json = json.loads(response.model_dump_json())
    input_tokens = response_json['usage']['prompt_tokens']
    cached_input_tokens = response_json['usage']['prompt_tokens_details']['cached_tokens']
    output_tokens = response_json['usage']['completion_tokens']

    prices = get_openai_model_price(response_json['model'])
    input_token_price_per_million_usd = prices['input_token_price_per_million_usd']
    cached_input_token_price_per_million_usd = prices['cached_input_token_price_per_million_usd']
    output_token_price_per_million_usd = prices['output_token_price_per_million_usd']

    million = 1000000
    input_tokens_cost = (input_tokens/million)*input_token_price_per_million_usd
    cached_input_tokens_cost = (cached_input_tokens/million)*cached_input_token_price_per_million_usd
    output_token_cost = (output_tokens/million)*output_token_price_per_million_usd

    total_cost = input_tokens_cost + cached_input_tokens_cost + output_token_cost
    output = {
        'input': {
            'rate': input_token_price_per_million_usd,
            'tokens': input_tokens,
            'cost': f'{input_tokens_cost:.10f}'
        },
        'cached_input': {
            'rate': cached_input_token_price_per_million_usd,
            'tokens': cached_input_tokens,
            'cost': f'{cached_input_tokens_cost:.10f}'
        },
        'output': {
            'rate': output_token_price_per_million_usd,
            'tokens': output_tokens,
            'cost': f'{output_token_cost:.10f}'
        },
        'total': {
            'cost': f'{total_cost:.10f}'
        }
    }
    return output


import ipywidgets as widgets
from IPython.display import display, HTML
def display_stats(stats):
    data = stats
    # Create an HTML formatted string for a compact table layout
    html_content = f"""
    <style>
        .usage-table {{
            font-family: Arial, sans-serif;
            font-size: 12px;
            width: 70%;
            border-collapse: collapse;
            margin: 10px auto;
        }}
        .usage-table th, .usage-table td {{
            padding: 6px 10px;
            text-align: left;
            border-bottom: 1px solid #ddd;
        }}
        .usage-table th {{
            background-color: #007bff;
            color: white;
            text-align: center;
        }}
        .input-row {{ background-color: #e3f2fd; }}
        .cached-row {{ background-color: #f1f8e9; }}
        .output-row {{ background-color: #ffebee; }}
        .total-row {{ background-color: #007bff; font-weight: bold; }}
    </style>
    
    <table class="usage-table">
        <tr>
            <th>Type</th>
            <th>Rate ($/1M)</th>
            <th>Tokens</th>
            <th>Cost ($)</th>
        </tr>
        <tr class="input-row">
            <td>Input 🔽</td>
            <td>{data['input']['rate']}</td>
            <td>{data['input']['tokens']}</td>
            <td>{data['input']['cost']}</td>
        </tr>
        <tr class="cached-row">
            <td>Cached Input 🔄</td>
            <td>{data['cached_input']['rate']}</td>
            <td>{data['cached_input']['tokens']}</td>
            <td>{data['cached_input']['cost']}</td>
        </tr>
        <tr class="output-row">
            <td>Output 🔼</td>
            <td>{data['output']['rate']}</td>
            <td>{data['output']['tokens']}</td>
            <td>{data['output']['cost']}</td>
        </tr>
        <tr class="total-row">
            <td>Total 💰</td>
            <td>—</td>
            <td>—</td>
            <td>{data['total']['cost']}</td>
        </tr>
    </table>
    """
    
    # Display in Jupyter Notebook
    display(HTML(html_content))

def display_response(response):
    display(JSON(json.loads(response.model_dump_json())))
    stats = get_chat_completion_stats(response)
    display_stats(stats)

In [3]:
# create tools for gpt

# we have themealdb apis to provide us information about various food categories, food ingredients, search meal by main-ingredient, meal images, meal preparation description

import requests

class TheMealDBAPI:
    def __init__(self, api_key):
        self.base_url = f"https://www.themealdb.com/api/json/v2/{api_key}"
    
    def get_all_food_categories(self):
        response = requests.get(f'{self.base_url}/list.php?c=list').json()
        categories = [item['strCategory'] for item in response['meals']]
        return categories

    def get_all_food_ingredients(self):
        response = requests.get(f'{self.base_url}/list.php?i=list').json()
        ingredients = [item['strIngredient'] for item in response['meals']]
        return ingredients

    def filter_meal_by_ingredient(self, ingredients):
        response = requests.get(f'{self.base_url}/filter.php?i={",".join(ingredients)}').json()
        return response

    def get_meal_details_by_id(self, meal_id):
        response = requests.get(f'{self.base_url}/lookup.php?i={meal_id}').json()
        return response

mealdb_api_key = None
with open('../MEALDB_API_KEY.txt') as file:
    mealdb_api_key = file.readline()
themealdbapi = TheMealDBAPI(mealdb_api_key)

# create function map
def call_function(name, args):

    if(name == "get_all_food_categories"):
        return themealdbapi.get_all_food_categories()
    if(name == "get_all_food_ingredients"):
        return themealdbapi.get_all_food_ingredients()

def get_function_schema(name):
    function_schema_map = {
        "get_all_food_categories": {
            "name": "get_all_food_categories",
            "description": "get all food categories, like vegetarian, non-vegetarian, etc.",
            "strict": True,
            "parameters": {
                "type": "object",
                "properties": {},
                "additionalProperties": False
              }
        },
        "get_all_food_ingredients": {
            "name": "get_all_food_ingredients",
            "description": "get a list of food ingredients to be used to prepare a meal",
            "strict": True,
            "parameters": {
                "type": "object",
                "properties": {},
                "additionalProperties": False
            }
        }
    }
    return function_schema_map[name]

themealdb_tools = []
themealdb_tools.append({
    "type": "function",
    "function": get_function_schema("get_all_food_categories")
})
themealdb_tools.append({
    "type": "function",
    "function": get_function_schema("get_all_food_ingredients")
})

In [4]:
def get_chat_model_completions(messages, tools=None):
    response = openai.chat.completions.create(
        model = 'gpt-4o-mini',
        messages = messages,
        temperature = 0,
        tools = tools
    )
    display_response(response)
    finish_reason = response.choices[0].finish_reason
    response_message = response.choices[0].message
    if(finish_reason == 'stop'):
        return response_message.content
    if(finish_reason == 'tool_calls'):
        print('will make tool calls')
        tool_calls = response_message.tool_calls

        if(tool_calls):
            messages.append(response_message.model_dump())
        
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_arguments = tool_call.function.arguments
                print(f'Function: {function_name}')
                print(f'Arguments: {function_arguments}')
                function_args = json.loads(function_arguments)
                function_response = call_function(function_name, function_args)
                print(f'Function Response: {function_response}')
        
                messages.append({
                    "role": "tool",
                    "name": function_name,
                    "content": str(function_response),
                    "tool_call_id": tool_call.id
                })
        return get_chat_model_completions(messages, tools)

test_messages = [
    {'role': 'system', 'content': 'You are a joke teller, you give responses in less than 40 words'},
    {'role': 'user', 'content': 'tell me a joke'}
]

print(get_chat_model_completions(test_messages))

# if we see a response, it means the api-key is still working

def is_flagged_text(user_input):
    response = openai.moderations.create(input=user_input)
    flagged = response.results[0].flagged
    return flagged

def append_message_to_conversation(conversation_log, message, role):
    conversation_log.append({"role": role, "content": message})

<IPython.core.display.JSON object>

Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,30,0.0000045000
Cached Input 🔄,0.075,0,0.0000000000
Output 🔼,0.6,18,0.0000108000
Total 💰,—,—,0.0000153000


Why did the scarecrow win an award? Because he was outstanding in his field!


In [5]:
conversation_log = []

def initialize_conversation():
    sample_food_categories = "Vegan, Vegetarian, etc."
    sample_food_ingredient = "Tofu, Rice, Chicken, etc."
    
    sample_output = {
           'meal_name': 'Grilled Chicken Quinoa Bowl',
           'nutrients': 'High in protein, fiber, and healthy fats',
           'ingredients': ['Chicken Breast',
            'Quinoa',
            'Avocado',
            'Spinach',
            'Cherry Tomatoes',
            'Olive Oil'],
           'main_ingredient': 'Chicken Breast',
           'food_category': 'Main Course',
           'description': 'This meal is packed with protein from the chicken, which helps in muscle recovery and strength gain. Quinoa provides complex carbohydrates for sustained energy, while spinach and tomatoes add essential vitamins.'
        }
    
    delimiter = "######"
    prompt = f"""

    You are an expert nutritionist, an expert meal planner and an expert health coach.
    Your objective is to interview the user and understand their health objective and present them with relevant meals.

    {delimiter}
    While you are interviewing the user, ask about their food categories.
    Before you ask the user about their food categories, get all food categories. You will be penalized if you start your conversation before getting all the food categories.
    You will add the list of food ingredients in the final output. The final output should contain food ingredients from authentic sources, so get all food ingredients. You will be penalized if you start your conversation before getting all the food ingredients.
    Once you know about all the food categories and food ingredients, keep it to yourself and do not show in the output for the users to choose from. Your conversation should be natural. Pick up a few random food categories and a few random food ingredient to show as an example to the user.conversation_log
    For example, when you show the food categories, you can show like {sample_food_categories}, and when you want to take examples of food ingredient, you can show something like {sample_food_ingredient}. Never show the entire list.
    If the user asks for more options, you can then share a few more varied options. But you will never show the entire list of food categories and food ingredients.

    {delimiter}
    The final output should contain meal name, which nutrients it contains, list of all the ingredients, name of the main ingredient, food category and a description of connection of health objective and the ingredients.
    The ingredient should only be from the list of ingredients that you fetched. The main_ingredient should be from the list of ingredients that you fetched.
    The final output should contain 5 meals only in a json format without any code markers. If you do not show 5 meals, you will be heavily penalized.
    the final output will look like {sample_output}

    {delimiter}
    Start by greeting the user and introducing yourself.
    
    """

    prompt_with_role = {"role": "system", "content": prompt}
    return prompt_with_role

In [6]:
conversation_log.append(initialize_conversation())
response = get_chat_model_completions(conversation_log, themealdb_tools)
print(response)
append_message_to_conversation(conversation_log, response, 'assistant')

<IPython.core.display.JSON object>

Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,598,0.0000897000
Cached Input 🔄,0.075,0,0.0000000000
Output 🔼,0.6,45,0.0000270000
Total 💰,—,—,0.0001167000


will make tool calls
Function: get_all_food_categories
Arguments: {}
Function Response: ['Beef', 'Breakfast', 'Chicken', 'Dessert', 'Goat', 'Lamb', 'Miscellaneous', 'Pasta', 'Pork', 'Seafood', 'Side', 'Starter', 'Vegan', 'Vegetarian']
Function: get_all_food_ingredients
Arguments: {}
Function Response: ['Chicken', 'Salmon', 'Beef', 'Pork', 'Avocado', 'Apple Cider Vinegar', 'Asparagus', 'Aubergine', 'Baby Plum Tomatoes', 'Bacon', 'Baking Powder', 'Balsamic Vinegar', 'Basil', 'Basil Leaves', 'Basmati Rice', 'Bay Leaf', 'Bay Leaves', 'Beef Brisket', 'Beef Fillet', 'Beef Gravy', 'Beef Stock', 'Bicarbonate Of Soda', 'Biryani Masala', 'Black Pepper', 'Black Treacle', 'Borlotti Beans', 'Bowtie Pasta', 'Bramley Apples', 'Brandy', 'Bread', 'Breadcrumbs', 'Broccoli', 'Brown Lentils', 'Brown Rice', 'Brown Sugar', 'Butter', 'Cacao', 'Cajun', 'Canned Tomatoes', 'Cannellini Beans', 'Cardamom', 'Carrots', 'Cashew Nuts', 'Cashews', 'Caster Sugar', 'Cayenne Pepper', 'Celeriac', 'Celery', 'Celery Salt', 

<IPython.core.display.JSON object>

Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,3584,0.0005376000
Cached Input 🔄,0.075,3456,0.0002592000
Output 🔼,0.6,105,0.0000630000
Total 💰,—,—,0.0008598000


Hello! I'm an expert nutritionist, meal planner, and health coach. I'm here to help you achieve your health objectives through tailored meal suggestions. 

To get started, could you please share your health goals? Are you looking to lose weight, gain muscle, maintain a healthy lifestyle, or something else? Additionally, do you have any dietary preferences or restrictions, such as vegetarian, vegan, or specific food allergies? 

Once I have that information, I can provide you with meal options that align with your goals!


In [8]:
prompt = 'Hi, I am Suyash. I play football. I want to lose weight, gain strength and improve my running endurance and stamina'
append_message_to_conversation(conversation_log, prompt, 'user')
response = get_chat_model_completions(conversation_log)
print(response)
append_message_to_conversation(conversation_log, response, 'assistant')

<IPython.core.display.JSON object>

Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,3660,0.0005490000
Cached Input 🔄,0.075,0,0.0000000000
Output 🔼,0.6,100,0.0000600000
Total 💰,—,—,0.0006090000


Great to meet you, Suyash! It's fantastic that you're focused on improving your strength and endurance for football. To help you achieve your goals of losing weight, gaining strength, and enhancing your running stamina, I’d like to know about your food preferences. 

Do you have any specific dietary categories you follow, such as vegan, vegetarian, or do you eat meat? Also, are there any particular foods you love or dislike? This will help me tailor the meal suggestions to your taste!


In [8]:
prompt = 'I am a vegetarian'
append_message_to_conversation(conversation_log, prompt, 'user')
response = get_chat_model_completions(conversation_log)
print(response)
append_message_to_conversation(conversation_log, response, 'assistant')

<IPython.core.display.JSON object>

Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,3744,0.0005616000
Cached Input 🔄,0.075,0,0.0000000000
Output 🔼,0.6,89,0.0000534000
Total 💰,—,—,0.0006150000


Thank you for sharing that you're a vegetarian! Now that I know your dietary preference, I can suggest meals that align with your vegetarian lifestyle. 

Let me provide you with a selection of vegetarian meals that are nutritious and delicious. Here are a few examples of ingredients I might use: Quinoa, Tofu, and Spinach. 

Now, I'll create a list of meals tailored to your vegetarian diet. Please hold on for a moment!


In [9]:
prompt = 'can you give the response in proper json format'
append_message_to_conversation(conversation_log, prompt, 'user')
response = get_chat_model_completions(conversation_log)
print(response)
append_message_to_conversation(conversation_log, response, 'assistant')

<IPython.core.display.JSON object>

Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,3849,0.0005773500
Cached Input 🔄,0.075,3712,0.0002784000
Output 🔼,0.6,609,0.0003654000
Total 💰,—,—,0.0012211500


{
  "meals": [
    {
      "meal_name": "Quinoa and Black Bean Salad",
      "nutrients": "High in protein, fiber, and antioxidants",
      "ingredients": ["Quinoa", "Black Beans", "Cherry Tomatoes", "Cucumber", "Red Onion", "Lime Juice"],
      "main_ingredient": "Quinoa",
      "food_category": "Salad",
      "description": "This salad is packed with protein from quinoa and black beans, providing essential amino acids. The fresh vegetables add vitamins and minerals, while lime juice enhances flavor and aids digestion."
    },
    {
      "meal_name": "Vegetable Stir-Fry with Tofu",
      "nutrients": "Rich in protein, vitamins, and minerals",
      "ingredients": ["Tofu", "Broccoli", "Bell Peppers", "Carrots", "Soy Sauce", "Garlic"],
      "main_ingredient": "Tofu",
      "food_category": "Main Course",
      "description": "This stir-fry is a great source of plant-based protein from tofu, combined with a variety of colorful vegetables that provide essential nutrients and antioxidant

In [10]:
def intent_confirmation(response_mealcoach_assistant):
    delimiter = "#######"
    prompt = f"""
        You are a senior evaluator and a python expert.
        You will go through the text and find out if it contains a list of meals.
        Each and every meal item should contain the keys meal_name, nutrients, ingredients, main_ingredient, food_category, description.

        {delimiter}
        You all the keys are definitely present, then and only then you will respond with a "yes". Otherwise you will respond with a "no"

        {delimiter}
        Here is your input: {response_mealcoach_assistant}
        
    """

    conversation = [{"role": "system", "content": prompt}]
    response = get_chat_model_completions(conversation)
    return response

In [20]:
response

'{\n  "meals": [\n    {\n      "meal_name": "Grilled Chicken Quinoa Bowl",\n      "nutrients": "High in protein, fiber, and healthy fats",\n      "ingredients": [\n        "Chicken Breast",\n        "Quinoa",\n        "Avocado",\n        "Spinach",\n        "Cherry Tomatoes",\n        "Olive Oil"\n      ],\n      "main_ingredient": "Chicken Breast",\n      "food_category": "Main Course",\n      "description": "This meal is packed with protein from the chicken, which helps in muscle recovery and strength gain. Quinoa provides complex carbohydrates for sustained energy, while spinach and tomatoes add essential vitamins."\n    },\n    {\n      "meal_name": "Salmon and Asparagus",\n      "nutrients": "Rich in omega-3 fatty acids, protein, and vitamins",\n      "ingredients": [\n        "Salmon",\n        "Asparagus",\n        "Olive Oil",\n        "Lemon",\n        "Garlic"\n      ],\n      "main_ingredient": "Salmon",\n      "food_category": "Main Course",\n      "description": "Salmon is

In [11]:
intent_confirmation(response)

<IPython.core.display.JSON object>

Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,715,0.0001072500
Cached Input 🔄,0.075,0,0.0000000000
Output 🔼,0.6,2,0.0000012000
Total 💰,—,—,0.0001084500


'yes'

In [12]:
def extract_python_dictionary(text):
    prompt = f"""

    You are a python expert.
    Your job is to extract python dictionary from {text} that contains the list of meals
    Each and every meal item contains the keys meal_name, nutrients, ingredients, main_ingredient, food_category, description.

    The output should be a json string without any code markers
    
    """

    conversation = [{"role": "system", "content": prompt}]
    response = get_chat_model_completions(conversation)
    return json.loads(response)

In [25]:
some_response = f"""
Thank you for that information, Suyash! Since you don't have specific dietary preferences, I can suggest a variety of meals that will support your goals of weight loss, strength gain, and improved endurance. 

Here are a few examples of food categories that I can include in your meal plan: 
- Main Course
- Side Dishes
- Snacks

Now, let me provide you with some meal options that are nutritious and aligned with your objectives. 

1. {{'meal_name': 'Grilled Chicken Quinoa Bowl', 'nutrients': 'High in protein, fiber, and healthy fats', 'ingredients': ['Chicken Breast', 'Quinoa', 'Avocado', 'Spinach', 'Cherry Tomatoes', 'Olive Oil'], 'main_ingredient': 'Chicken Breast', 'food_category': 'Main Course', 'description': 'This meal is packed with protein from the chicken, which helps in muscle recovery and strength gain. Quinoa provides complex carbohydrates for sustained energy, while spinach and tomatoes add essential vitamins.'}}

2. {{'meal_name': 'Salmon and Asparagus', 'nutrients': 'Rich in omega-3 fatty acids, protein, and vitamins', 'ingredients': ['Salmon', 'Asparagus', 'Olive Oil', 'Lemon', 'Garlic'], 'main_ingredient': 'Salmon', 'food_category': 'Main Course', 'description': 'Salmon is an excellent source of protein and omega-3 fatty acids, which are beneficial for heart health and muscle recovery. Asparagus adds fiber and essential vitamins, making this a balanced meal for endurance athletes.'}}

3. {{'meal_name': 'Chickpea Salad', 'nutrients': 'High in protein, fiber, and antioxidants', 'ingredients': ['Chickpeas', 'Cucumber', 'Tomatoes', 'Red Onion', 'Olive Oil', 'Lemon Juice'], 'main_ingredient': 'Chickpeas', 'food_category': 'Side', 'description': 'This refreshing salad is packed with protein from chickpeas and is rich in fiber, which aids in digestion and keeps you full longer. The fresh vegetables provide essential vitamins and minerals for overall health.'}}

4. {{'meal_name': 'Vegetable Stir-Fry with Tofu', 'nutrients': 'High in protein, vitamins, and minerals', 'ingredients': ['Tofu', 'Broccoli', 'Bell Peppers', 'Carrots', 'Soy Sauce', 'Ginger'], 'main_ingredient': 'Tofu', 'food_category': 'Main Course', 'description': 'Tofu is a great source of plant-based protein, and the colorful vegetables provide a variety of nutrients. This stir-fry is low in calories but high in flavor, making it perfect for weight loss and muscle gain.'}}

5. {{'meal_name': 'Oatmeal with Berries', 'nutrients': 'High in fiber, antioxidants, and healthy carbohydrates', 'ingredients': ['Rolled Oats', 'Blueberries', 'Almonds', 'Honey', 'Milk'], 'main_ingredient': 'Rolled Oats', 'food_category': 'Breakfast', 'description': 'Oatmeal is a fantastic source of complex carbohydrates that provide sustained energy for your workouts. Topped with berries and almonds, this meal is rich in antioxidants and healthy fats, supporting your overall health and performance.'}}

These meals are designed to provide you with the nutrients you need to support your training and help you reach your goals. If you have any questions or need more options, feel free to ask!"
"""

In [15]:
response

'{\n  "meals": [\n    {\n      "meal_name": "Quinoa and Black Bean Salad",\n      "nutrients": "High in protein, fiber, and antioxidants",\n      "ingredients": ["Quinoa", "Black Beans", "Cherry Tomatoes", "Cucumber", "Red Onion", "Lime Juice"],\n      "main_ingredient": "Quinoa",\n      "food_category": "Salad",\n      "description": "This salad is packed with protein from quinoa and black beans, providing essential amino acids. The fresh vegetables add vitamins and minerals, while lime juice enhances flavor and aids digestion."\n    },\n    {\n      "meal_name": "Vegetable Stir-Fry with Tofu",\n      "nutrients": "Rich in protein, vitamins, and minerals",\n      "ingredients": ["Tofu", "Broccoli", "Bell Peppers", "Carrots", "Soy Sauce", "Garlic"],\n      "main_ingredient": "Tofu",\n      "food_category": "Main Course",\n      "description": "This stir-fry is a great source of plant-based protein from tofu, combined with a variety of colorful vegetables that provide essential nutrient

In [16]:
meals_suggesed_by_assistant = extract_python_dictionary(response)
meals_suggesed_by_assistant

<IPython.core.display.JSON object>

Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,678,0.0001017000
Cached Input 🔄,0.075,0,0.0000000000
Output 🔼,0.6,609,0.0003654000
Total 💰,—,—,0.0004671000


{'meals': [{'meal_name': 'Quinoa and Black Bean Salad',
   'nutrients': 'High in protein, fiber, and antioxidants',
   'ingredients': ['Quinoa',
    'Black Beans',
    'Cherry Tomatoes',
    'Cucumber',
    'Red Onion',
    'Lime Juice'],
   'main_ingredient': 'Quinoa',
   'food_category': 'Salad',
   'description': 'This salad is packed with protein from quinoa and black beans, providing essential amino acids. The fresh vegetables add vitamins and minerals, while lime juice enhances flavor and aids digestion.'},
  {'meal_name': 'Vegetable Stir-Fry with Tofu',
   'nutrients': 'Rich in protein, vitamins, and minerals',
   'ingredients': ['Tofu',
    'Broccoli',
    'Bell Peppers',
    'Carrots',
    'Soy Sauce',
    'Garlic'],
   'main_ingredient': 'Tofu',
   'food_category': 'Main Course',
   'description': 'This stir-fry is a great source of plant-based protein from tofu, combined with a variety of colorful vegetables that provide essential nutrients and antioxidants.'},
  {'meal_name

In [22]:
meals = []
for meal_suggested in meals_suggesed_by_assistant['meals']:
    main_ingredient = meal_suggested['main_ingredient']
    print(main_ingredient)
    meals.extend(themealdbapi.filter_meal_by_ingredient([main_ingredient])['meals'])

Quinoa
Tofu
Spinach
Chickpeas
Spinach


In [23]:
meals

[{'strMeal': 'Chicken Quinoa Greek Salad',
  'strMealThumb': 'https://www.themealdb.com/images/media/meals/k29viq1585565980.jpg',
  'idMeal': '53011'},
 {'strMeal': 'Stuffed Bell Peppers with Quinoa and Black Beans',
  'strMealThumb': 'https://www.themealdb.com/images/media/meals/b66myb1683207208.jpg',
  'idMeal': '53067'},
 {'strMeal': 'Hot and Sour Soup',
  'strMealThumb': 'https://www.themealdb.com/images/media/meals/1529445893.jpg',
  'idMeal': '52954'},
 {'strMeal': 'Ma Po Tofu',
  'strMealThumb': 'https://www.themealdb.com/images/media/meals/1525874812.jpg',
  'idMeal': '52947'},
 {'strMeal': 'Potato Gratin with Chicken',
  'strMealThumb': 'https://www.themealdb.com/images/media/meals/qwrtut1468418027.jpg',
  'idMeal': '52780'},
 {'strMeal': 'Salmon Avocado Salad',
  'strMealThumb': 'https://www.themealdb.com/images/media/meals/1549542994.jpg',
  'idMeal': '52960'},
 {'strMeal': 'Tuna and Egg Briks',
  'strMealThumb': 'https://www.themealdb.com/images/media/meals/2dsltq1560461468

In [25]:
# index = 4
# main_ingredient = meals[index]['main_ingredient']
main_ingredient = 'Bell Peppers'
themealdbapi.filter_meal_by_ingredient([main_ingredient])

{'meals': None}

In [130]:
themealdbapi.get_meal_details_by_id(52869)

{'meals': [{'idMeal': '52869',
   'strMeal': 'Tahini Lentils',
   'strMealAlternate': None,
   'strCategory': 'Vegetarian',
   'strArea': 'Moroccan',
   'strInstructions': 'In a jug, mix the tahini with the zest and juice of the lemon and 50ml of cold water to make a runny dressing. Season to taste, then set aside.\r\nHeat the oil in a wok or large frying pan over a medium-high heat. Add the red onion, along with a pinch of salt, and fry for 2 mins until starting to soften and colour. Add the garlic, pepper, green beans and courgette and fry for 5 min, stirring frequently.\r\nTip in the kale, lentils and the tahini dressing. Keep the pan on the heat for a couple of mins, stirring everything together until the kale is wilted and it’s all coated in the creamy dressing.',
   'strMealThumb': 'https://www.themealdb.com/images/media/meals/vpxyqt1511464175.jpg',
   'strTags': 'Pulse',
   'strYoutube': 'https://www.youtube.com/watch?v=8kRlmz8pW0I',
   'strIngredient1': 'Tahini',
   'strIngredi